connect to the database

In [140]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Define credentials and parameters
HOST = "localhost"
PORT = "6543"
USER = "postgres"
PASSWORD = "postgres"
DBNAME = "evision_01"

# Connect to PostgreSQL without a specific database
connection = psycopg2.connect(
    host=HOST, 
    port=PORT, 
    user=USER, 
    password=PASSWORD
)
connection.autocommit = True
cursor = connection.cursor()

# Check if the database exists, and if not, create it
cursor.execute("SELECT datname FROM pg_database;")
db_list = [db[0] for db in cursor.fetchall()]

if DBNAME not in db_list:
    cursor.execute(f"CREATE DATABASE {DBNAME};")

cursor.close()
connection.close()

# Create an SQLAlchemy engine for the "Evision" database
engine_url = f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}'
engine = create_engine(engine_url)

In [175]:
import os
folder_name = 'data'

create Token entity

In [176]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer

# Connect to the database
metadata = MetaData()

# Read the "data/token.csv" CSV file into a pandas DataFrame
df = pd.read_csv(os.path.join(folder_name, 'token.csv'))

# Define the Token table schema with 'token_address' as the primary key
token = Table('Token', metadata,
    Column('token_address', String, primary_key=True),
    Column('decimals', Integer),
    Column('logo', String),
    Column('name', String),
    Column('symbol', String)
)

# Drop the existing Token table if it exists, and then create a new one
token.drop(engine, checkfirst=True)
token.create(engine)

# Populate the Token table using the DataFrame
df.to_sql("Token", engine, if_exists="append", index=False)


14

create User entity

In [177]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String, PrimaryKeyConstraint

# Connect to the database
metadata = MetaData()

# Read the "data/user.csv" CSV file into a pandas DataFrame
df = pd.read_csv(os.path.join(folder_name, 'user.csv'))

# Define the User table schema with 'ethereum_address' as the primary key
user = Table('User', metadata,
    Column('ethereum_address', String, primary_key=True)
)

# Drop the existing User table if it exists, and then create a new one
user.drop(engine, checkfirst=True)
user.create(engine)

# Populate the User table using the DataFrame
df.to_sql("User", engine, if_exists="append", index=False)


616

create LiquidityPool entity

In [178]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, ForeignKey

# 2. Define the LiquidityPool table schema with 'liquidity_pool_address' as the primary key
liquidity_pool = Table('LiquidityPool', metadata,
    Column('liquidity_pool_address', String, primary_key=True),
    # Column('token0_address', String, ForeignKey('Token.token_address')),
    # Column('token1_address', String, ForeignKey('Token.token_address')),
    Column('token0_address', String),
    Column('token1_address', String),
    Column('sqrtPriceX96', Float),
    Column('liquidity', Float)
)

# 3. Drop the existing LiquidityPool table if it exists
liquidity_pool.drop(engine, checkfirst=True)

# 4. Create the LiquidityPool table
liquidity_pool.create(engine)

# 5. Read the "data/liquidity_pool.csv" CSV file into a pandas DataFrame
df = pd.read_csv(os.path.join(folder_name, 'liquidity_pool.csv'))

# Populate the LiquidityPool table using the DataFrame
df.to_sql("LiquidityPool", engine, if_exists="append", index=False)

855

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, ForeignKey

# 2. Define the LiquidityPool table schema with 'liquidity_pool_address' as the primary key
liquidity_pool = Table('LiquidityPool', metadata,
    Column('liquidity_pool_address', String, primary_key=True),
    # Column('token0_address', String, ForeignKey('Token.token_address')),
    # Column('token1_address', String, ForeignKey('Token.token_address')),
    Column('token0_address', String),
    Column('token1_address', String),
    Column('sqrtPriceX96', Float),
    Column('liquidity', Float)
)

# 3. Drop the existing LiquidityPool table if it exists
liquidity_pool.drop(engine, checkfirst=True)

# 4. Create the LiquidityPool table
liquidity_pool.create(engine)

# 5. Read the "data/liquidity_pool.csv" CSV file into a pandas DataFrame
df = pd.read_csv(os.path.join(folder_name, 'liquidity_pool.csv'))

# Populate the LiquidityPool table using the DataFrame
df.to_sql("LiquidityPool", engine, if_exists="append", index=False)

create Swap associative entity.

In [179]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey, BIGINT

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(os.path.join(folder_name, 'uniswap-v3-swap.csv'))

# Drop the 'block_number' column
df = df.drop(columns=['block_number'])

# Define the Swap table schema
swap = Table('Swap', metadata,
    Column('timestamp', String),
    Column('tx_hash', String, primary_key=True),
    Column('log_index', Integer),
    Column('pool_contract_address', String, ForeignKey('LiquidityPool.liquidity_pool_address')),
    Column('amount0', String),
    Column('amount1', String),
    Column('sqrt_price_x96', String),
    Column('liquidity', String),
    Column('tick', String),
    Column('ethereum_address', String, ForeignKey('User.ethereum_address'))
)

swap.drop(engine, checkfirst=True)

swap.create(engine)

df.to_sql("Swap", engine, if_exists="append", index=False)

65

create CreatePool entity.

In [180]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey, BIGINT

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(os.path.join(folder_name, 'uniswap-v3-poolcreated.csv'))

# Drop the 'block_number' column
df = df.drop(columns=['block_number', 'token0_symbol', 'token1_symbol'])
df['pool_contract_address'] = df['pool_contract_address'].str.lower()

# Define the Swap table schema
CreatePool = Table('CreatePool', metadata,
    Column('timestamp', String),
    Column('tx_hash', String, primary_key=True),
    Column('log_index', Integer),
    Column('factory_contract_address', String),
    Column('pool_contract_address', String, ForeignKey('LiquidityPool.liquidity_pool_address')),
    Column('fee', Integer),
    Column('token0_address', String),
    Column('token1_address', String),
    Column('ethereum_address', String, ForeignKey('User.ethereum_address'))
)

CreatePool.drop(engine, checkfirst=True)

CreatePool.create(engine)

df.to_sql("CreatePool", engine, if_exists="append", index=False)

855

create mint entity

In [181]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey, BIGINT

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(os.path.join(folder_name, 'uniswap-v3-mint.csv'))

# Drop the 'block_number' column
df = df.drop(columns=['block_number'])
df['pool_contract_address'] = df['pool_contract_address'].str.lower()

# Define the Swap table schema
Mint = Table('Mint', metadata,
    Column('timestamp', String),
    Column('tx_hash', String, primary_key=True),
    Column('log_index', Integer),
    Column('pool_contract_address', String, ForeignKey('LiquidityPool.liquidity_pool_address')),
    Column('tick_lower', String),
    Column('tick_upper', String),
    Column('amount', String),
    Column('amount0', String),
    Column('amount1', String),
    Column('ethereum_address', String, ForeignKey('User.ethereum_address'))
)

Mint.drop(engine, checkfirst=True)

Mint.create(engine)

df.to_sql("Mint", engine, if_exists="append", index=False)

77

create burn entity

In [182]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey, BIGINT

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(os.path.join(folder_name, 'uniswap-v3-burn.csv'))

# Drop the 'block_number' column
df = df.drop(columns=['block_number'])
df['pool_contract_address'] = df['pool_contract_address'].str.lower()

# Define the Swap table schema
Burn = Table('Burn', metadata,
    Column('timestamp', String),
    Column('tx_hash', String, primary_key=True),
    Column('log_index', Integer),
    Column('pool_contract_address', String, ForeignKey('LiquidityPool.liquidity_pool_address')),
    Column('tick_lower', String),
    Column('tick_upper', String),
    Column('amount', String),
    Column('amount0', String),
    Column('amount1', String),
    Column('ethereum_address', String, ForeignKey('User.ethereum_address'))
)

Burn.drop(engine, checkfirst=True)

Burn.create(engine)

df.to_sql("Burn", engine, if_exists="append", index=False)

619

In [183]:
# CreatePool.drop(engine, checkfirst=True)
# swap.drop(engine, checkfirst=True)
# Burn.drop(engine, checkfirst=True)
# Mint.drop(engine, checkfirst=True)